<!-- Auto table of contents -->
<h1 class='tocIgnore'>AutoDS : Validation tests</h1>
<p>(for the <b>autods</b> module, a python interface to MCDS.exe, http://distancesampling.org/)</p>
<div style="overflow-y: auto">
  <h2 class='tocIgnore'>Table of contents</h2>
  <div id="toc"></div>
</div>

In [ ]:
%%javascript
$.getScript('ipython_notebook_toc.js')

In [ ]:
%matplotlib inline

In [ ]:
import sys
import os
import pathlib as pl
import importlib as implib

import re

from collections import OrderedDict as odict, namedtuple as ntuple

import math
import numpy as np
import pandas as pd

from IPython.display import HTML

import matplotlib.pyplot as plt

import plotly as ply
import plotly.graph_objs as plygo

In [ ]:
sys.path.insert(0, '..')

In [ ]:
import autods as ads 

In [ ]:
# Logging configuration.
ads.logger('ads', level=ads.log.INFO, handlers=[sys.stdout, 'tmp/valtst.log'], verbose=True)
ads.logger('ads.rep', level=ads.log.INFO2)
ads.logger('ads.opn', level=ads.log.DEBUG)
ads.logger('ads.opr', level=ads.log.DEBUG)

logger = ads.logger('valtst', level=ads.log.DEBUG, handlers=[sys.stdout, 'tmp/valtst.log'], verbose=True)

In [ ]:
# Activate Warnings as Exceptions
if False:
    
    import warnings

    warnings.filterwarnings(action='error')

    # pd.read_excel
    warnings.filterwarnings(action='default', module='etree')
    warnings.filterwarnings(action='default', module='xlrd')
    warnings.filterwarnings(action='default', module='defusedxml')

# I. Run analyses with real life field data (1/2 : long code, long run)

With MCDSAnalysis class.

(for comparison to manually issued analyses with Distance 7.3)

## 1. Load analyses set specifications

In [ ]:
# Load refout results table
refFileName = 'ACDC2019-Papyrus-ALAARV-TURMER-resultats-distance-73.xlsx'
dfRefRes = pd.read_excel(pl.Path('refout', refFileName))
dfRefRes.reset_index(inplace=True) # Generate analysis # (later need for original cases order)
dfRefRes.rename(columns=dict(index='AnlysNum', Name='Model'), inplace=True)

dfRefRes.head()

## 2. Build test cases

In [ ]:
# Generate test cases definition code from refout results file (don't cheat : only input columns :-)
modelIdCols = ['Model']
modelParamCols = ['LTrunc', 'RTrunc', 'FitDistCuts', 'DiscrDistCuts']
sampleIdCols = ['Species', 'Periods', 'Prec.', 'Duration']
caseIdCols = ['AnlysNum'] + sampleIdCols + modelIdCols
dfAnlysCases = dfRefRes[caseIdCols + modelParamCols].copy()

dfAnlysCases['KeyFn'] = \
    dfAnlysCases.Model.apply(lambda s: 'UNIFORM' if s.startswith('Unif') \
                                                 else 'HNORMAL' if s.startswith('Half') else 'HAZARD')
dfAnlysCases['AdjSer'] = \
    dfAnlysCases.Model.apply(lambda s: 'COSINE' if s.find(' Cos') > 0 \
                                                else 'POLY' if s.find(' SimPoly') > 0 else 'HERMITE')
dfAnlysCases['InFileName'] = \
    dfAnlysCases.apply(lambda sRow: 'ACDC2019-Papyrus-{}-{}-{}mn-{}dec-dist.txt' \
                                    .format(sRow.Species,
                                            'AB' if 'A+B' in sRow.Periods else 'A' if 'A' in sRow.Periods else 'B',
                                            sRow.Duration.split(' ')[0], sRow['Prec.'].split(' ')[0]),
                       axis='columns')
dfAnlysCases

In [ ]:
#def nan2None(v):
#    return None if pd.isnull(v) else v
def distCutsFromSpecs(v):
    if pd.isnull(v):
        return None
    if isinstance(v, int):
        return v
    return [float(x) for x in v.split(',')]

## 3. Prepare analyses

In [ ]:
decimalFields = ['Point transect*Survey effort', 'Observation*Radial distance']

In [ ]:
# Analysis engine (sequential)
mcds = ads.MCDSEngine(workDir=pl.Path('tmp', 'mcds-out'),
                      executor=None, # Non-parallel: ~7.5s elapsed on a Lenovo P52 (6-core i7-8850H with PCI-e SSD)
                      distanceUnit='Meter', areaUnit='Hectare',
                      surveyType='Point', distanceType='Radial')

In [ ]:
# Frozen analysis parameters (a choice here)
KEstimCriterion = 'AIC'
KCVInterval = 95

In [ ]:
# Results object construction
sampCols = [('sample', col, 'Value') for col in sampleIdCols]
miSampCols = pd.MultiIndex.from_tuples(sampCols)

custCols = [('sample', 'AnlysNum', 'Value')] + sampCols + [('model', 'Model', 'Value')]
miCustCols = pd.MultiIndex.from_tuples(custCols)

dfCustColTrans = \
    pd.DataFrame(index=miCustCols,
                 data=dict(en=caseIdCols, 
                           fr=['NumAnlys', 'Espèce', 'Périodes', 'Préc.', 'Durée', 'Modèle']))

results = ads.MCDSAnalysisResultsSet(miCustomCols=miCustCols, miSampleCols=miSampCols, dfCustomColTrans=dfCustColTrans)

In [ ]:
computed = False

## 3a. Or : Really run analyses

In [ ]:
# Shorten test cases and reference results lists, to go faster
# Warning: If you don't retain entire samples, later comparison will fail on Delta AIC values.
#selCaseInds = [0, 5, 7, 22, 31] # Some random cases, with uncomplete samples.
#selCaseInds = dfAnlysCases[dfAnlysCases.Sample.isin([3, 4])].index # A shorter selection, with complete samples.
selCaseInds = range(len(dfAnlysCases)) # All of them.

nOrigAnlysCases = len(dfAnlysCases)
dfAnlysCases = dfAnlysCases.loc[selCaseInds]
dfRefRes = dfRefRes.loc[selCaseInds]

logger.info(f'Retained {len(selCaseInds)} out of {nOrigAnlysCases}.')

In [ ]:
%%time

# Run all analyses
lastInFileName = None
for _, sCase in dfAnlysCases.iterrows():
    
    nCase = sCase.AnlysNum
    name = sCase.InFileName[len('ACDC2019-Papyrus')+1:-len('-dist.txt')]
    name += '-' + sCase.Model.lower().translate(str.maketrans({c:'-' for c in ' ,.:;()/'}))
    logger.info(f'#{nCase+1:3d} {name} {sCase.KeyFn} {sCase.AdjSer}')
    
    # Create data set if not already done.
    if lastInFileName != sCase.InFileName:
        sds = ads.SampleDataSet(pl.Path('refin', sCase.InFileName), decimalFields=decimalFields)
        lastInFileName = sCase.InFileName
        
    # Run analysis and get results
    anlys = ads.MCDSAnalysis(engine=mcds, sampleDataSet=sds, name=name, logData=True,
                             estimKeyFn=sCase.KeyFn, estimAdjustFn=sCase.AdjSer,
                             estimCriterion=KEstimCriterion, cvInterval=KCVInterval,
                             minDist=sCase.LTrunc, maxDist=sCase.RTrunc,
                             fitDistCuts=distCutsFromSpecs(sCase.FitDistCuts),
                             discrDistCuts=distCutsFromSpecs(sCase.DiscrDistCuts))

    anlys.submit()
    sResult = anlys.getResults()

    # Save results
    sHead = pd.Series(data=[sCase[col] for col in sCase.index[:len(caseIdCols)]], index=miCustCols)

    results.append(sResult, sCustomHead=sHead)
    
# shutdown analysis engine
mcds.shutdown()

# Done.
computed = True

In [ ]:
# Save results in case need for not recomputing them
resFileName = os.path.join(mcds.workDir, 'autods-validation-results.xlsx')

results.toExcel(resFileName, sheetName='AutoDSVal')

In [ ]:
resFileName = os.path.join(mcds.workDir, 'autods-validation-results-en.xlsx')

results.toExcel(resFileName, sheetName='Auto', lang='en')

In [ ]:
# Check translation
dfActTrRes = results.dfTransData('fr')

dfActTrRes.head().T.iloc[:30] #.at['TroncGche', 0]

## 3b. Or : Load analyses from a previous run

(already run and saved above)

In [ ]:
if not computed:
    
    resFileName = os.path.join(mcds.workDir, 'autods-validation-results.xlsx')
    print('Loading results from {} ...'.format(resFileName))

    results.fromExcel(resFileName, sheetName='AutoDSVal')
    
    # shutdown analysis engine
    mcds.shutdown()

else:
    
    print('Just computed, not reloading ...')
    
print('... {} analyses to compare'.format(len(results)))

# II. Compare actual results to reference

(reference = manually run analyses with Distance software)

## 1. Extract actual results to compare

In [ ]:
# Analysis results
dfActRes = results.dfData

dfActRes.head().T[:30]

In [ ]:
# Sélection des colonnes des résultats autos et association aux disponibles dans la référence, pour comparaison.
dCompCols = \
{
    ('sample', 'AnlysNum', 'Value'):  'AnlysNum',
    ('sample', 'Species', 'Value'):   'Species',
    ('sample', 'Periods', 'Value'):   'Periods',
    ('sample', 'Prec.', 'Value'):     'Prec.',
    ('sample', 'Duration', 'Value'):  'Duration',
    
    ('model',  'Model', 'Value'):         'Model',
    ('parameters', 'left truncation distance', 'Value'):           'LTrunc',
    ('parameters', 'right truncation distance', 'Value'):          'RTrunc',
    ('parameters', 'model fitting distance cut points', 'Value'):  'FitDistCuts',
    ('parameters', 'distance discretisation cut points', 'Value'): 'DiscrDistCuts',
    
    ('run output', 'run status', 'Value') : 'Status',
    #('run output', 'run time', 'Value') : 'Run', # Only for unintests ref. generation just below
    
    ('detection probability', 'total number of parameters (m)', 'Value'): '# params',
    ('encounter rate', 'number of observations (n)', 'Value'): '# obs',
    
    ('detection probability', 'Delta AIC', 'Value'): 'Delta AIC',
    ('detection probability', 'AIC value', 'Value'): 'AIC',
    ('detection probability', 'chi-square test probability determined', 'Value')               : 'GOF Chi-p',
    ('detection probability', 'Kolmogorov-Smirnov test probability', 'Value')                  : 'GOF K-S p',
    ('detection probability', 'Cramér-von Mises (uniform weighting) test probability', 'Value'): 'GOF CvM (unif) p',
    ('detection probability', 'Cramér-von Mises (cosine weighting) test probability', 'Value') : 'GOF CvM (cos) p',
    
    ('detection probability', 'effective strip width (ESW) or effective detection radius (EDR)', 'Value'): 'ESW/EDR',
    ('detection probability', 'effective strip width (ESW) or effective detection radius (EDR)', 'Lcl')  : 'ESW/EDR LCL',
    ('detection probability', 'effective strip width (ESW) or effective detection radius (EDR)', 'Ucl')  : 'ESW/EDR UCL',
    ('detection probability', 'effective strip width (ESW) or effective detection radius (EDR)', 'Cv')   : 'ESW/EDR CV',
    
    ('density/abundance', 'density of animals', 'Value'): 'D',
    ('density/abundance', 'density of animals', 'Lcl')  : 'D LCL',
    ('density/abundance', 'density of animals', 'Ucl')  : 'D UCL',
    ('density/abundance', 'density of animals', 'Cv')   : 'D CV',
    
    ('detection probability', 'probability of detection (Pw)', 'Value'): 'P',
    ('detection probability', 'probability of detection (Pw)', 'Lcl')  : 'P LCL',
    ('detection probability', 'probability of detection (Pw)', 'Ucl')  : 'P UCL',
    ('detection probability', 'probability of detection (Pw)', 'Cv')   : 'P CV',
    ('detection probability', 'probability of detection (Pw)', 'Df')   : 'P DF',
}
len(dCompCols)

In [ ]:
# Warning: Unused columns (full of NaNs) have been atomatically removed
# (see last line of AnalysisResultsSet.dfData getter)
dCompCols = { k: v for k, v in dCompCols.items() if k in dfActRes.columns }
len(dCompCols)

In [ ]:
# So we need to cleanup modelParamCols too
modelParamCols = [id_ for id_ in modelParamCols if id_ in dCompCols.values()]
len(modelParamCols)

In [ ]:
# Safe stringification of model params
# * needed for use in indexes (hashability)
# * needed to cope with to_excel/read_excel unconsistent None management
def modelParam2Str(par):
    #print(par)
    if isinstance(par, list):
        spar = str([float(v) for v in par])
    elif pd.isnull(par):
        spar = 'None'
    elif isinstance(par, str):
        if ',' in par: # Assumed already somewhat stringified list
            spar = str([float(v) for v in par.strip('[]').split(',')])
    else:
        spar = str(par)
    return spar

In [ ]:
# Select results columns and rename them as reference, for easier comparison
dfActRes4c = dfActRes[list(dCompCols.keys())].copy()
dfActRes4c.columns = [dCompCols[col] for col in dCompCols]
dfActRes4c[modelParamCols] = dfActRes4c[modelParamCols].applymap(modelParam2Str) # Hashable mandatory for indexing
dfActRes4c.set_index(caseIdCols + modelParamCols, inplace=True)

dfActRes4c

In [ ]:
# Select usefull reference columns for comarison
dfRefRes4c = dfRefRes.copy()
dfRefRes4c[modelParamCols] = dfRefRes4c[modelParamCols].applymap(modelParam2Str) # Hashable mandatory for indexing
dfRefRes4c.set_index(caseIdCols + modelParamCols, inplace=True)
dfRefRes4c = dfRefRes4c.reindex(columns=dfActRes4c.columns)

dfRefRes4c

## 2. Diagnostic automatique

In [ ]:
# First checks : equality of test case lists (index) and of column names (columns)
assert sorted(dfActRes4c.index)   == sorted(dfRefRes4c.index)
assert sorted(dfActRes4c.columns) == sorted(dfRefRes4c.columns)

In [ ]:
# Actual / reference closeness measure : -round(log10((actual - reference) / max(abs(actual), abs(reference))), 1)
# = Compute the order of magnitude that separate the difference from the absolute max. of the two values
# The greater it is, the lower the relative difference
#    Ex: 3 = 10**3 ratio between difference absolue max. of the two,
#        +inf = NO difference at all,
#        0 = bad, one of the two is 0, and the other not,
# See unitary test below.
def closeness(sRefAct):
    
    x, y = sRefAct.to_list()
    
    # Special cases with 1 NaN, or 1 or more inf => all different
    if np.isnan(x):
        if not np.isnan(y):
            return 0 # All different
    elif np.isnan(y):
        return 0 # All different
    
    if np.isinf(x) or np.isinf(y):
        return 0 # All different
    
    # Normal case
    c = abs(x - y)
    if not np.isnan(c) and c != 0:
        c = c / max(abs(x), abs(y))
    
    return np.inf if c == 0 else round(-np.log10(c), 1)

In [ ]:
# Actual / reference comparison : compute closeness indicator
dfRelDif = dfRefRes4c.copy()
for col in dfRelDif.columns:
    dfRelDif['act'] = dfActRes4c[col]
    dfRelDif[col] = dfRelDif[[col, 'act']].apply(closeness, axis='columns')
    dfRelDif.drop(columns='act', inplace=True)
    
dfRelDif

In [ ]:
# Diagnosis : we only keep lines and columns with some relevant differences.
dfBadRelDif = dfRelDif.copy()
len(dfBadRelDif)

In [ ]:
# 1. Suppression lignes : Status identique et reste NaN (cas des status = 0/3/4 : erreur d'exécution, ou pas d'exécution)
valCols = [col for col in dfRelDif.columns if col != 'Status']
dfBadRelDif.drop(dfBadRelDif[(dfBadRelDif.Status.abs() == np.inf) & dfBadRelDif[valCols].isnull().all(axis='columns')].index,
                 axis='index', inplace=True)
assert len(dfBadRelDif) == 29, len(dfBadRelDif)
len(dfBadRelDif)

In [ ]:
# 2. Suppression lignes : Status et toutes autres colonnes à inf (stricte égalité)
#    NB. Somme very small differences observed when results have just been computed or when they have been
#        loaded from a previously saved Excel file (above 10**15 closeness value)
dfBadRelDif.drop(dfBadRelDif[dfBadRelDif.apply(np.isinf, axis='columns').all(axis='columns')].index,
                 axis='index', inplace=True)
assert (computed and len(dfBadRelDif) == 26) or (not computed and len(dfBadRelDif) == 19), len(dfBadRelDif)
len(dfBadRelDif)

In [ ]:
# 3. Suppression lignes : Status et toutes autres colonnes supérieures à 15 (quasi stricte égalité)
dfBadRelDif.drop(dfBadRelDif[(dfBadRelDif >= 15).all(axis='columns')].index, axis='index', inplace=True)
assert len(dfBadRelDif) == 9, len(dfBadRelDif)
len(dfBadRelDif)

In [ ]:
# 4. Suppression lignes : Status identique et toutes autres colonnes supérieures à 4 (quasi égalité)
dfBadRelDif.drop(dfBadRelDif[(dfBadRelDif >= 4).all(axis='columns')].index, axis='index', inplace=True)
assert len(dfBadRelDif) == 4, len(dfBadRelDif)
len(dfBadRelDif)

In [ ]:
# 5. Suppression lignes : Status identique et toutes autres colonnes supérieures à 4 (quasi égalité)
#                         sauf colonnes GOF KS et CvM à NaN, non calculées quand on discrétise les distances.
if 'DiscrDistCuts' in dfBadRelDif.index.names:
    discrCols = [col for col in dfRelDif.columns if not col.startswith('GOF') or col.find('Chi') > 0]
    df2Drop = (dfBadRelDif.index.get_level_values('DiscrDistCuts') != -1) & (dfBadRelDif[discrCols] >= 4).all(axis='columns')
    dfBadRelDif.drop(dfBadRelDif[df2Drop].index, axis='index', inplace=True)
assert len(dfBadRelDif) == 2, len(dfBadRelDif)
len(dfBadRelDif)

In [ ]:
# Le verdict (Cf. fichier Excel refFileName, feuille "DiffAuto" pour explications des 2 différences Act/Ref)
dfBadRelDif.T

In [ ]:
dfRefRes4c.loc[dfBadRelDif.index]

In [ ]:
nFails = len(dfBadRelDif.index)
if nFails > 0:
    print(f'Warning: {nFails} test case(s) failed ;')
    print(f' ... see sheet "DiffAuto" of {refFileName} for possible explanations.')
else:
    print('All test cases succeeded !')

## 3. Save results to disk.

In [ ]:
resCompFileName = os.path.join(mcds.workDir, 'autods-validation-rescomp.xlsx')

with pd.ExcelWriter(resCompFileName) as xlsxWriter:

    dfRefRes.to_excel(xlsxWriter, sheet_name='RefResults', index=True)
    dfActRes4c.reset_index().to_excel(xlsxWriter, sheet_name='ActResults', index=False)
    dfRelDif.reset_index().to_excel(xlsxWriter, sheet_name='Diff2Ref', index=False)
    dfBadRelDif.reset_index().to_excel(xlsxWriter, sheet_name='BadDiff2Ref', index=False)
    dfRefRes4c.loc[dfBadRelDif.index].reset_index().to_excel(xlsxWriter, sheet_name='RefResWithDiff', index=False)
    dfActRes4c.loc[dfBadRelDif.index].reset_index().to_excel(xlsxWriter, sheet_name='ActResWithDiff', index=False)
    dfActRes.to_excel(xlsxWriter, sheet_name='RawActResults', index=True)

In [ ]:
dfActRes.head()

# III. Parallel run of same analyses

## 1. Prepare analyses

(same test cases and input data as previously, for easy comparison)

In [ ]:
# Analysis executor : 6, 8, None threads => min elapsed = ~2s on a Lenovo P52 (6-core i7-8850H with PCI-e SSD)
parallelExecutor = ads.Executor(parallel=True, threads=6)

# Analysis engine
mcds = ads.MCDSEngine(workDir=pl.Path('tmp', 'mcds-out'), executor=parallelExecutor, 
                      distanceUnit='Meter', areaUnit='Hectare',
                      surveyType='Point', distanceType='Radial')

In [ ]:
# Results object construction
parResults = ads.MCDSAnalysisResultsSet(miCustomCols=miCustCols, miSampleCols=miSampCols, dfCustomColTrans=dfCustColTrans)

In [ ]:
computed = False

## 2a. Or : Really run analyses

In [ ]:
# Shorten test cases and reference results lists, to go faster
# Warning: If you don't retain entire samples, later comparison will fail on Delta AIC values.
#selCaseInds = [0, 5, 7, 22, 31] # Some random cases, with uncomplete samples.
#selCaseInds = dfAnlysCases[dfAnlysCases.Sample.isin([3, 4])].index # A shorter selection, with complete samples.
selCaseInds = range(len(dfAnlysCases)) # All of them.

nOrigAnlysCases = len(dfAnlysCases)
dfAnlysCases = dfAnlysCases.loc[selCaseInds]
dfRefRes = dfRefRes.loc[selCaseInds]

logger.info(f'Retained {len(selCaseInds)} out of {nOrigAnlysCases}.')

In [ ]:
%%time

# Start running all analyses
lastInFileName = None
analyses = dict()
for _, sCase in dfAnlysCases.iterrows():
    
    nCase = sCase.AnlysNum
    name = sCase.InFileName[len('ACDC2019-Papyrus')+1:-len('-dist.txt')]
    name += '-' + sCase.Model.lower().translate(str.maketrans({c:'-' for c in ' ,.:;()/'}))
    logger.info(f'#{nCase+1:3d} {name} {sCase.KeyFn} {sCase.AdjSer}')
    
    # Create data set if not already done.
    if lastInFileName != sCase.InFileName:
        sds = ads.SampleDataSet(pl.Path('refin', sCase.InFileName), decimalFields=decimalFields)
        lastInFileName = sCase.InFileName
        
    # Start running analysis in parallel (don't wait for it's finished, go on)
    sResHead = pd.Series(data=[sCase[col] for col in sCase.index[:len(caseIdCols)]], index=miCustCols)

    anlys = ads.MCDSAnalysis(engine=mcds, sampleDataSet=sds, name=name, customData=sResHead, logData=True,
                             estimKeyFn=sCase.KeyFn, estimAdjustFn=sCase.AdjSer,
                             estimCriterion=KEstimCriterion, cvInterval=KCVInterval,
                             minDist=sCase.LTrunc, maxDist=sCase.RTrunc,
                             #minDist=nan2None(sCase.LTrunc), maxDist=nan2None(sCase.RTrunc),
                             fitDistCuts=distCutsFromSpecs(sCase.FitDistCuts), # TODO: do this when building dfAnlysCases
                             discrDistCuts=distCutsFromSpecs(sCase.DiscrDistCuts))
    anlysFut = anlys.submit()
    
    # Store analysis object and associated "future" for later use (should be running soon or later).
    analyses[anlysFut] = anlys
    
logger.info('All analyses started ; now waiting for their end, and results ...')

# For each analysis as it gets completed (first completed => first yielded)
for anlysFut in parallelExecutor.asCompleted(analyses):

    # Retrieve analysis object from its associated future object
    anlys = analyses[anlysFut]
    
    # Get analysis results
    sResult = anlys.getResults()

    # Save results with header
    parResults.append(sResult, sCustomHead=anlys.customData)
    
# shutdown analysis engine
mcds.shutdown()

# Done.
computed = True

In [ ]:
# Save results in case need for not recomputing them
resFileName = os.path.join(mcds.workDir, 'autods-validation-parallel-results.xlsx')

parResults.toExcel(resFileName, sheetName='AutoDSVal')

## 2b. Or : Load analyses from a previous run

(already run and saved above)

In [ ]:
if not computed:
    
    resFileName = os.path.join(mcds.workDir, 'autods-validation-parallel-results.xlsx')
    print('Loading results from {} ...'.format(resFileName))

    parResults.fromExcel(resFileName, sheetName='AutoDSVal')
    
    # shutdown analysis engine
    mcds.shutdown()

else:
    
    print('Just computed, not reloading ...')
    
print(f'... {len(parResults)} analyses to compare')

## 3. Compare parallel results to sequential ones

In [ ]:
# Prepare sequential results for comparison
dfSeqCmpRes = results.dfTransData('en')

dfSeqCmpRes.fillna(-9999, inplace=True) # Get rid of the Nan pb (because NaN != NaN :-)

dfSeqCmpRes.drop(columns=['RunTime', 'RunFolder'], inplace=True) # Run date-time and folder can never be the same

In [ ]:
# Prepare parallel results for comparison
dfParCmpRes = parResults.dfTransData('en')

dfParCmpRes.sort_values(by='AnlysNum', inplace=True) # Back to original test case order = sequential run order

dfParCmpRes.reset_index(inplace=True, drop=True) # Enforce same index as a consequence

dfParCmpRes.fillna(-9999, inplace=True) # And get rid of the Nan pb (because NaN != Nan :-)

dfParCmpRes.drop(columns=['RunTime', 'RunFolder'], inplace=True) # Run date-time and folder can never be the same

In [ ]:
assert (dfSeqCmpRes == dfParCmpRes).all().all(), 'Oh, oh, something went differently when run parallely ...'

In [ ]:
results = parResults

# IV. Excel and HTML reports

In [ ]:
# Sélection des colonnes pour les tableaux de synthèse du rapport
synthCols = \
[
    ('sample', 'AnlysNum', 'Value'),
    ('sample', 'Species', 'Value'),
    ('sample', 'Periods', 'Value'),
    ('sample', 'Prec.', 'Value'),
    ('sample', 'Duration', 'Value'),
    
    ('model', 'Model', 'Value'),
    ('parameters', 'left truncation distance', 'Value'),
    ('parameters', 'right truncation distance', 'Value'),
    ('parameters', 'model fitting distance cut points', 'Value'),
    ('parameters', 'distance discretisation cut points', 'Value'),
    
    ('run output', 'run status', 'Value'),
    
    ('encounter rate', 'number of observations (n)', 'Value'),
    
    ('detection probability', 'Delta AIC', 'Value'),
    ('detection probability', 'AIC value', 'Value'),
    ('detection probability', 'chi-square test probability determined', 'Value'),
    ('detection probability', 'Kolmogorov-Smirnov test probability', 'Value'),
    ('density/abundance', 'density of animals', 'Cv'),
    
    ('detection probability', 'effective strip width (ESW) or effective detection radius (EDR)', 'Value'),
    ('detection probability', 'effective strip width (ESW) or effective detection radius (EDR)', 'Lcl'),
    ('detection probability', 'effective strip width (ESW) or effective detection radius (EDR)', 'Ucl'),
    
    ('density/abundance', 'density of animals', 'Value'),
    ('density/abundance', 'density of animals', 'Lcl'),
    ('density/abundance', 'density of animals', 'Ucl'),
    
    ('detection probability', 'probability of detection (Pw)', 'Value'),
    ('detection probability', 'probability of detection (Pw)', 'Lcl'),
    ('detection probability', 'probability of detection (Pw)', 'Ucl'),
    ('detection probability', 'probability of detection (Pw)', 'Df'),

    ('run output', 'run folder', 'Value'),
]

In [ ]:
report = ads.MCDSResultsFullReport(resultsSet=results, synthCols=synthCols, title='Validation du module autods',
                                   subTitle='Rapport d\'analyse global', anlysSubTitle='Rapport détaillé',
                                   description='Qu\'ajouter de plus ?', keywords='autods, validation',
                                   lang='fr', attachedDir='.', tgtFolder=mcds.workDir, tgtPrefix='autods-validation-report')

In [ ]:
htmlRep = report.toHtml()

HTML(f'Rapport HTML : <a href="{htmlRep}" target="blank">{htmlRep}</a>')

In [ ]:
xlsxRep = report.toExcel()

HTML(f'Rapport Excel : <a href="{xlsxRep}" target="blank">{xlsxRep}</a>')

# V. Run and report pre-analyses (1/3 : long code, long duration)

(to help users to setup the full analyses plan : run first try simple analyses and show PDF and few results)

## 1. Determine samples from input data

* in real life, we'd simply load field collected data, and deduce individual "samples" from it ;
* but there, for testing, it's easier to deduce samples from manual analysis specification file)

In [ ]:
# Create sample table from refout results table
refFileName = 'ACDC2019-Papyrus-ALAARV-TURMER-resultats-distance-73.xlsx'

sampleIdCols = ['Species', 'Periods', 'Prec.', 'Duration']

dfSamples = pd.read_excel(pl.Path('refout', refFileName), usecols=sampleIdCols)
dfSamples.rename(columns=dict(Name='Model'), inplace=True)
dfSamples.drop_duplicates(inplace=True)
dfSamples.reset_index(drop=True, inplace=True)

dfSamples.reset_index(inplace=True) # Generate sample # (later need for original sample order)
dfSamples.rename(columns=dict(index='SampleNum'), inplace=True)

sampleIdCols = ['SampleNum'] + sampleIdCols

dfSamples

## 2. Prepare analyses

In [ ]:
decimalFields = ['Point transect*Survey effort', 'Observation*Radial distance']

In [ ]:
# Analysis engine: Non-parallel executor here, 'cause MCDSPreAnalysis takes care of this !
mcds = ads.MCDSEngine(workDir=pl.Path('tmp', 'mcds-pout'), # 
                      distanceUnit='Meter', areaUnit='Hectare',
                      surveyType='Point', distanceType='Radial')

In [ ]:
# Results object construction
custCols = [('sample', col, 'Value') for col in sampleIdCols]
miCustCols = pd.MultiIndex.from_tuples(custCols)
dfCustColTrans = \
    pd.DataFrame(index=miCustCols,
                 data=dict(en=sampleIdCols, 
                           fr=['NumEchant', 'Espèce', 'Périodes', 'Préc.', 'Durée']))

results = ads.MCDSAnalysisResultsSet(miCustomCols=miCustCols, dfCustomColTrans=dfCustColTrans)

In [ ]:
computed = False

## 3 Or : Really run pre-analyses

In [ ]:
KPreEstimCrit = 'AIC'
KPreCVInterval = 95
KPreEstimModStrat = [dict(keyFn=kf, adjSr='COSINE', estCrit=KPreEstimCrit, cvInt=KPreCVInterval) \
                     for kf in['HNORMAL', 'HAZARD', 'UNIFORM', 'NEXPON']]

In [ ]:
%%time

# Run all analyses
lastInFileName = None
for _, sSamp in dfSamples.iterrows():
    
    nSamp = sSamp.SampleNum
    sampId = '{}-{}-{}mn-{}dec' \
             .format(sSamp.Species,
                     'AB' if 'A+B' in sSamp.Periods else 'A' if 'A' in sSamp.Periods else 'B',
                     sSamp.Duration.split(' ')[0], sSamp['Prec.'].split(' ')[0])
    logger.info(f'#{nSamp+1:3d} {sampId}')
    
    # Create data set if not already done.
    inFileName = 'ACDC2019-Papyrus-{}-dist.txt'.format(sampId)
    if lastInFileName != inFileName:
        sds = ads.SampleDataSet(pl.Path('refin', inFileName), decimalFields=decimalFields)
        lastInFileName = inFileName
        
    # Run analysis: Not parallel runs for now ... see below.
    preAnlys = ads.MCDSPreAnalysis(engine=mcds, sampleDataSet=sds, name=sampId, executor=None,
                                   logData=False, modelStrategy=KPreEstimModStrat)
    preAnlys.submit()
    
    # Get results (wait for it's finished)
    sResult = preAnlys.getResults()

    # Save results
    sResHead = sSamp.copy()
    sResHead.index = miCustCols
    results.append(sResult, sCustomHead=sResHead)
    
# shutdown analysis engine
mcds.shutdown()

# Done.
computed = True

In [ ]:
# Look at results
results.dfTransData('fr')[['NumEchant', 'Espèce', 'Périodes', 'Préc.', 'Durée', 'Fn Clé',
                           'Sér Ajust', 'CodEx', 'NObs', 'AIC', 'Chi2 P', 'KS P', 
                           'Densité', 'CoefVar Densité', 'Min Densité', 'Max Densité']]

In [ ]:
# Save results in case need for not recomputing them
resFileName = os.path.join(mcds.workDir, 'autods-validation-preresults.xlsx')

results.toExcel(resFileName, sheetName='AutoDSVal')

## 3. Or : Load analyses from a previous run

(already run and saved above)

In [ ]:
if not computed:
    
    resFileName = os.path.join(mcds.workDir, 'autods-validation-preresults.xlsx')
    print(f'Loading pre-results from {resFileName} ...')

    results.fromExcel(resFileName, sheetName='AutoDSVal')
    
else:
    
    print('Just computed, not reloading ...')
    
print(f'... {len(results)} pre-analyses loaded')

In [ ]:
# Look at results
results.dfTransData('fr')[['NumEchant', 'Espèce', 'Périodes', 'Préc.', 'Durée', 'Fn Clé',
                           'Sér Ajust', 'CodEx', 'NObs', 'AIC', 'Chi2 P', 'KS P', 
                           'Densité', 'CoefVar Densité', 'Min Densité', 'Max Densité']]

## 4. Generate HTML and Excel reports

In [ ]:
# Sélection des colonnes pour les tableaux de synthèse du rapport
synthCols = \
[
    ('sample', 'SampleNum', 'Value'),
    ('sample', 'Species', 'Value'),
    ('sample', 'Periods', 'Value'),
    ('sample', 'Prec.', 'Value'),
    ('sample', 'Duration', 'Value'),
    
    ('parameters', 'estimator key function', 'Value'),
    ('parameters', 'estimator adjustment series', 'Value'),
    ('parameters', 'CV interval', 'Value'),
    ('parameters', 'left truncation distance', 'Value'),
    ('parameters', 'right truncation distance', 'Value'),
    ('parameters', 'model fitting distance cut points', 'Value'),
    ('parameters', 'distance discretisation cut points', 'Value'),
    
    ('run output', 'run status', 'Value'),
    
    ('encounter rate', 'number of observations (n)', 'Value'),
    
    ('detection probability', 'AIC value', 'Value'),
    ('detection probability', 'chi-square test probability determined', 'Value'),
    ('detection probability', 'Kolmogorov-Smirnov test probability', 'Value'),
    ('density/abundance', 'density of animals', 'Cv'),
    
    ('detection probability', 'effective strip width (ESW) or effective detection radius (EDR)', 'Value'),
    ('detection probability', 'effective strip width (ESW) or effective detection radius (EDR)', 'Lcl'),
    ('detection probability', 'effective strip width (ESW) or effective detection radius (EDR)', 'Ucl'),
    
    ('density/abundance', 'density of animals', 'Value'),
    ('density/abundance', 'density of animals', 'Lcl'),
    ('density/abundance', 'density of animals', 'Ucl'),
    
    ('detection probability', 'probability of detection (Pw)', 'Value'),
    ('detection probability', 'probability of detection (Pw)', 'Lcl'),
    ('detection probability', 'probability of detection (Pw)', 'Ucl'),
    ('detection probability', 'probability of detection (Pw)', 'Df'),

    ('run output', 'run folder', 'Value'),
]

In [ ]:
# Select analysis results columns for the 3 textual columns of the synthesis pre-report
sampleCols = \
[
    ('sample', 'SampleNum', 'Value'),
    ('sample', 'Species', 'Value'),
    ('sample', 'Periods', 'Value'),
    ('sample', 'Prec.', 'Value'),
    ('sample', 'Duration', 'Value')
]

paramCols = \
[
    ('parameters', 'estimator key function', 'Value'),
    ('parameters', 'estimator adjustment series', 'Value'),
    ('parameters', 'CV interval', 'Value')
]
    
resultCols = \
[
    ('run output', 'run status', 'Value'),
    
    ('encounter rate', 'number of observations (n)', 'Value'),
    
    ('detection probability', 'AIC value', 'Value'),
    ('detection probability', 'chi-square test probability determined', 'Value'),
    ('detection probability', 'Kolmogorov-Smirnov test probability', 'Value'),
    ('detection probability', 'probability of detection (Pw)', 'Value'),
    ('detection probability', 'effective strip width (ESW) or effective detection radius (EDR)', 'Value'),

    ('density/abundance', 'density of animals', 'Cv'),
    ('density/abundance', 'density of animals', 'Value'),
    ('density/abundance', 'density of animals', 'Lcl'),
    ('density/abundance', 'density of animals', 'Ucl'),
]

In [ ]:
report = ads.MCDSResultsPreReport(resultsSet=results,
                                  title='Validation du module autods', subTitle='Rapport de pré-analyse',
                                  anlysSubTitle='Détail des pré-analyses', description='Qu\'ajouter de plus ?',
                                  keywords='autods, validation', plotsHeight=384, lang='fr',
                                  sampleCols=sampleCols, paramCols=paramCols, resultCols=resultCols, anlysSynthCols=synthCols,
                                  attachedDir='.', tgtFolder=mcds.workDir, tgtPrefix='autods-validation-prereport')

In [ ]:
htmlRep = report.toHtml()

HTML(f'Pré-rapport HTML : <a href="{htmlRep}" target="blank">{htmlRep}</a>')

In [ ]:
xlsxRep = report.toExcel()

HTML(f'Rapport Excel : <a href="{xlsxRep}" target="blank">{xlsxRep}</a>')

# VI. Parallel run of same pre-analyses (2/3 : long code, short duration)

And compare results to sequential run's.

## 1. Prepare analyses

In [ ]:
# Analysis engine : Non-parallel executor here: MCDSPreAnalysis takes care of this !
mcds = ads.MCDSEngine(workDir=pl.Path('tmp', 'mcds-ppout'),
                      distanceUnit='Meter', areaUnit='Hectare',
                      surveyType='Point', distanceType='Radial')

In [ ]:
# Results object construction
parResults = ads.MCDSAnalysisResultsSet(miCustomCols=miCustCols, dfCustomColTrans=dfCustColTrans)

In [ ]:
computed = False

## 2 Or : Really run pre-analyses

In [ ]:
# Pre-analysis executor (kind of overkill here, with only 5 pre-analyses ... but still works twice as rapidly !).
parallelExecutor = ads.Executor(parallel=True, threads=6)

In [ ]:
%%time

# Run all analyses
lastInFileName = None
preAnalyses = dict()
for _, sSamp in dfSamples.iterrows():
    
    nSamp = sSamp.SampleNum
    sampId = '{}-{}-{}mn-{}dec' \
             .format(sSamp.Species,
                     'AB' if 'A+B' in sSamp.Periods else 'A' if 'A' in sSamp.Periods else 'B',
                     sSamp.Duration.split(' ')[0], sSamp['Prec.'].split(' ')[0])
    logger.info(f'#{nSamp+1:3d} {sampId}')
    
    # Create data set if not already done.
    inFileName = 'ACDC2019-Papyrus-{}-dist.txt'.format(sampId)
    if lastInFileName != inFileName:
        sds = ads.SampleDataSet(pl.Path('refin', inFileName), decimalFields=decimalFields)
        lastInFileName = inFileName
        
    # Start running analysis (but don't wait for it's finished)
    sResHead = sSamp.copy()
    sResHead.index = miCustCols
    
    preAnlys = ads.MCDSPreAnalysis(engine=mcds, sampleDataSet=sds, name=sampId,
                                   customData=sResHead, executor=parallelExecutor,
                                   logData=False, modelStrategy=KPreEstimModStrat)
    preAnlysFut = preAnlys.submit()
    
    # Store analysis object and associated "future" for later use (should be running soon or later).
    preAnalyses[preAnlysFut] = preAnlys
    
logger.info('All pre-analyses started ; now waiting for their end, and results ...')

# For each analysis as it gets completed (first completed => first yielded)
for preAnlysFut in parallelExecutor.asCompleted(preAnalyses):

    # Retrieve pre-analysis object from its associated future object
    preAnlys = preAnalyses[preAnlysFut]
    
    # Get pre-analysis results
    sResult = preAnlys.getResults()

    # Save results with header
    parResults.append(sResult, sCustomHead=preAnlys.customData)
    
# shutdown executor
parallelExecutor.shutdown()

# shutdown analysis engine
mcds.shutdown()

# Done.
computed = True

In [ ]:
# Look at results
parResults.dfTransData('fr')[['NumEchant', 'Espèce', 'Périodes', 'Préc.', 'Durée', 'Fn Clé',
                              'Sér Ajust', 'CodEx', 'NObs', 'AIC', 'Chi2 P', 'KS P', 
                              'Densité', 'CoefVar Densité', 'Min Densité', 'Max Densité']]

In [ ]:
# Save results in case need for not recomputing them
resFileName = os.path.join(mcds.workDir, 'autods-validation-preresults-par.xlsx')

parResults.toExcel(resFileName, sheetName='AutoDSVal')

## 2. Or : Load analyses from a previous run

(already run and saved above)

In [ ]:
if not computed:
    
    resFileName = os.path.join(mcds.workDir, 'autods-validation-preresults-par.xlsx')
    print('Loading pre-results from {} ...'.format(resFileName))

    parResults.fromExcel(resFileName, sheetName='AutoDSVal')
    
else:
    
    print('Just computed, not reloading ...')
    
print('... {} pre-analyses loaded'.format(len(parResults)))

In [ ]:
# Look at results
parResults.dfTransData('fr')[['NumEchant', 'Espèce', 'Périodes', 'Préc.', 'Durée', 'Fn Clé',
                              'Sér Ajust', 'CodEx', 'NObs', 'AIC', 'Chi2 P', 'KS P', 
                              'Densité', 'CoefVar Densité', 'Min Densité', 'Max Densité']]

## 3. Compare parallel results to sequential ones

In [ ]:
# Prepare sequential results for comparison
dfSeqCmpRes = results.dfTransData('en')

dfSeqCmpRes.fillna(-9999, inplace=True) # Get rid of the Nan pb (because NaN != NaN :-)

dfSeqCmpRes.drop(columns=['RunTime', 'RunFolder'], inplace=True) # Run date-time and folder can never be the same

In [ ]:
# Prepare parallel results for comparison
dfParCmpRes = parResults.dfTransData('en')

dfParCmpRes.sort_values(by='SampleNum', inplace=True) # Back to original test case order = sequential run order

dfParCmpRes.reset_index(inplace=True, drop=True) # Enforce same index as a consequence

dfParCmpRes.fillna(-9999, inplace=True) # And get rid of the Nan pb (because NaN != Nan :-)

dfParCmpRes.drop(columns=['RunTime', 'RunFolder'], inplace=True) # Run date-time and folder can never be the same

In [ ]:
assert (dfSeqCmpRes == dfParCmpRes).all().all(), 'Oh, oh, something went differently when run parallely ...'

# VII. Run analyses with real life field data (2/2 : short code + fast run)

Thanks to MCDSAnalyser class.

In [ ]:
transectPlaceCols = ['Point']
passIdCol = 'Passage'
effortCol = 'Effort'

sampleDecCols = [effortCol, 'Distance']

sampleNumCol = 'NumEchant'
sampleSelCols = ['Espèce', passIdCol, 'Adulte', 'Durée']

varIndCol = 'NumAnlys'
anlysAbbrevCol = 'AbrevAnlys'

dSurveyArea = dict(Zone='ACDC', Surface='2400')

## 1. Individuals data set

In [ ]:
dfObsIndiv = ads.DataSet('refin/ACDC2019-Naturalist-ExtraitObsIndiv.ods', sheet='DonnéesIndiv').dfData
len(dfObsIndiv)

In [ ]:
dfObsIndiv.head()

In [ ]:
{ col: dfObsIndiv[col].unique() for col in ['Observateur', 'Point', 'Passage', 'Adulte', 'Durée', 'Espèce'] }

## 2. Actual transects

(can't deduce them from data, some points are missing because of data selection)

In [ ]:
dfTransects = ads.DataSet('refin/ACDC2019-Naturalist-ExtraitObsIndiv.ods', sheet='Inventaires').dfData
len(dfTransects)

In [ ]:
dfTransects

## 3. Analyses specs

In [ ]:
def analysisAbbreviation(sAnlys):
    
    # Sample abbreviation
    spcAbbrev = ''.join(word[:4].title() for word in sAnlys['Espèce'].split(' ')[:2])
    sampAbbrev = [str(x) for x in [spcAbbrev, sAnlys.Passage.replace('+', ''),
                                   sAnlys.Adulte.replace('+', ''), sAnlys['Durée']]]

    # Model + Parameters abbreviation
    modParAbbrev = [sAnlys['FonctionClé'][:3].lower(), sAnlys['SérieAjust'][:3].lower()]
    if not pd.isnull(sAnlys['TrGche']):
        modParAbbrev.append('g{}'.format(int(sAnlys['TrGche'])))
    if not pd.isnull(sAnlys['TrDrte']):
        modParAbbrev.append('d{}'.format(int(sAnlys['TrDrte'])))
    if not pd.isnull(sAnlys['NbTrchMod']):
        modParAbbrev.append('t{}'.format(int(sAnlys['NbTrchMod'])))
    
    return '-'.join(sampAbbrev + modParAbbrev)

dfAnlysSpecs = ads.Analyser.explicitVariantSpecs('refin/ACDC2019-Naturalist-ExtraitSpecsAnalyses.xlsx', 
                                                 keep=['Echant1_impl', 'Echant2_impl', 'Modl_impl',
                                                       'Params1_expl', 'Params2_expl'],
                                                 varIndCol=varIndCol,
                                                 #convertCols={ 'Durée': int }, # float 'cause of Excel
                                                 computedCols={ anlysAbbrevCol: analysisAbbreviation })

len(dfAnlysSpecs)

In [ ]:
# For faster debugging : reduce work.
#dfAnlysSpecs = dfAnlysSpecs[(dfAnlysSpecs.Passage == 'a+b') & (dfAnlysSpecs.Adulte == 'm') \
#                            & (dfAnlysSpecs['Durée'] == '10mn') \
#                            & ((dfAnlysSpecs.TrGche.isnull()) | (dfAnlysSpecs.TrGche < 20)) \
#                            & ((dfAnlysSpecs.TrDrte.isnull()) | (dfAnlysSpecs.TrDrte <= 500))]
#len(dfAnlysSpecs)

In [ ]:
dfAnlysSpecs

In [ ]:
# Recall analysis set without truncation params
dfAnlysSpecs[['Espèce', 'Passage', 'Adulte', 'Durée', 'FonctionClé', 'SérieAjust']].drop_duplicates() \
        .reset_index(drop=True).reset_index()


In [ ]:
computed = False

In [ ]:
workDir = 'tmp/mcds-anlr'

## 4A. Or : Really run analyses

### a. MCDS Analyser object

In [ ]:
anlysr = ads.MCDSAnalyser(dfObsIndiv, dfTransects=dfTransects, dSurveyArea=dSurveyArea,
                          transectPlaceCols=transectPlaceCols, passIdCol=passIdCol, effortCol=effortCol,
                          sampleSelCols=sampleSelCols, sampleDecCols=sampleDecCols,
                          distanceUnit='Meter', areaUnit='Hectare',
                          surveyType='Point', distanceType='Radial', clustering=False,
                          resultsHeadCols=dict(before=[varIndCol, sampleNumCol], sample=sampleSelCols,
                                               after=[anlysAbbrevCol]),
                          abbrevCol=anlysAbbrevCol, anlysIndCol=varIndCol, sampleIndCol=sampleNumCol,
                          workDir=workDir,
                          defEstimCriterion='AIC', defCVInterval=95)

### b. Run analyses

In [ ]:
# Colonnes de dfFinalExplSpecs donnant les "vrais" paramètres d'analyse
#anlysParamSpecCols = ['FonctionClé', 'SérieAjust', 'TrGche', 'TrDrte', 'NbTrchMod']

In [ ]:
assert anlysr.checkUserSpecs(dfAnlysSpecs, anlysParamSpecCols)

In [ ]:
%%time

# Analyses : min=5, max=11s elapsed for 64 analyses on a Lenovo P52 (6-core i7-8850H with PCI-e SSD)
results = anlysr.run(dfAnlysSpecs, threads=6)

computed = True

In [ ]:
anlysr.shutdown()

In [ ]:
results.dfTransData('fr')

In [ ]:
results.dfData

### c. Save results for later reload or examination

In [ ]:
results.toExcel(pl.Path(workDir) / 'valtests-mcds-anlyser-results.xlsx')

In [ ]:
#results.toExcel(pl.Path(workDir) / 'valtests-mcds-anlyser-results-fr.xlsx', lang='fr')

## 4B. Or : Load analyses from a previous run

(already run and saved above)

In [ ]:
if not computed:
    
    # An analyser object knowns how to build an empty results object ...
    anlysr = ads.MCDSAnalyser(dfObsIndiv, dfTransects=dfTransects, dSurveyArea=dSurveyArea,
                              resultsHeadCols=dict(before=[varIndCol, sampleNumCol], sample=sampleSelCols,
                                                   after=[anlysAbbrevCol]),
                              transectPlaceCols=transectPlaceCols, passIdCol=passIdCol, effortCol=effortCol,
                              sampleSelCols=sampleSelCols, sampleDecCols=sampleDecCols,
                              distanceUnit='Meter', areaUnit='Hectare',
                              surveyType='Point', distanceType='Radial', clustering=False,
                              abbrevCol=anlysAbbrevCol, anlysIndCol=varIndCol, sampleIndCol=sampleNumCol,
                              workDir=workDir)
    
    results = anlysr.setupResults()
    
    anlysr.shutdown()
    
    # Load results from file.
    resFileName = pl.Path(workDir) / 'valtests-mcds-anlyser-results.xlsx'
    print('Loading results from {} ...'.format(resFileName))

    results.fromExcel(resFileName)
    
else:
    
    print('Just computed, not reloading ...')
    
print('... {} analyses to compare'.format(len(results)))

## 5. Compare results to reference

(reference generated with same kind of "long" code like in III above, but on another data set)

In [ ]:
# Load reference
# 1. Clone results _without_ data.
rsRef = results.copy(withData=False)

# 2. Load it with reference data
rsRef.fromOpenDoc('refout/ACDC2019-Naturalist-ExtraitResultats.ods')

rsRef.dfData

In [ ]:
# Compare (ignore sample and analysis indexes, no use here).
indexCols = [col for col in results.miCustomCols.to_list() if '(sample)' in col[0]] \
            + [('parameters', 'estimator key function', 'Value'),
               ('parameters', 'estimator adjustment series', 'Value'),
               ('parameters', 'left truncation distance', 'Value'),
               ('parameters', 'right truncation distance', 'Value'),
               ('parameters', 'model fitting distance cut points', 'Value')]
subsetCols = [col for col in results.dfData.columns.to_list() \
              if col not in (indexCols + [col for col in results.miCustomCols.to_list()
                                          if '(sample)' not in col[0]]
                             + [('parameters', 'estimator selection criterion', 'Value'),
                                ('parameters', 'CV interval', 'Value'),
                                ('run output', 'run time', 'Value'),
                                ('run output', 'run folder', 'Value'),
                                ('detection probability', 'key function type', 'Value'),
                                ('detection probability', 'adjustment series type', 'Value')])]

dfDiff = rsRef.compare(results, indexCols=indexCols, subsetCols=subsetCols, dropCloser=13, dropNans=True)

assert dfDiff.empty, 'No, no, no : not the same ...'

print('Yessssss !')

In [ ]:
# To be perfectly honnest ... some 10**-15 glitches (due to worksheet I/O ?)
rsRef.compare(results, indexCols=indexCols, subsetCols=subsetCols, dropCloser=14, dropNans=True)

# VIII. Run pre-analyses (3/3 : short code, short duration)

And compare results to sequential run's.

In [ ]:
transectPlaceCols = ['Point']
passIdCol = 'Passage'
effortCol = 'Effort'

sampleDecCols = [effortCol, 'Distance']

sampleNumCol = 'NumEchant'
sampleSelCols = ['Espèce', passIdCol, 'Adulte', 'Durée']

sampleAbbrevCol = 'AbrevEchant'

dSurveyArea = dict(Zone='ACDC', Surface='2400')

## 1. Individuals data set

In [ ]:
dfObsIndiv = ads.DataSet('refin/ACDC2019-Naturalist-ExtraitObsIndiv.ods', sheet='DonnéesIndiv').dfData
dfObsIndiv

In [ ]:
{ col: dfObsIndiv[col].unique() for col in ['Observateur', 'Point', 'Passage', 'Adulte', 'Durée', 'Espèce'] }

## 2. Actual transects

(can't deduce them from data, some points are missing because of data selection)

In [ ]:
dfTransects = ads.DataSet('refin/ACDC2019-Naturalist-ExtraitObsIndiv.ods', sheet='Inventaires').dfData
dfTransects

## 3. Samples to pre-analyse

In [ ]:
# Implicit variants
varEspeces = ['Sylvia atricapilla', 'Turdus merula', 'Luscinia megarhynchos'] # 1 variante espèce ... par espèce <8-]

varPassages = ['b', 'a+b'] # Passage b ou a+b => 2 variantes
varAdultes = ['m'] # Les mâles, et ensuite les mâles et autres adultes (=> 2 variantes)
varDurees = ['5mn', '10mn'] # 5 1ères mn, ou toutes les 10 => 2 variantes

In [ ]:
def sampleAbbreviation(sAnlys):
    
    # Sample abbreviation
    spcAbbrev = ''.join(word[:4].title() for word in sAnlys['Espèce'].split(' ')[:2])
    sampAbbrev = [str(x) for x in [spcAbbrev, sAnlys.Passage.replace('+', ''),
                                   sAnlys.Adulte.replace('+', ''), sAnlys['Durée']]]

    return '-'.join(sampAbbrev)

In [ ]:
# Explicitation of variants
# a. Implicites specs
dImplSampleSpecs = { 'Espèce': varEspeces, 'Passage': varPassages, 'Adulte': varAdultes, 'Durée':   varDurees }

# b. Explicit combinations
dfExplSampleSpecs = ads.Analyser.explicitVariantSpecs(dict(_impl=dImplSampleSpecs))
#dfExplSampleSpecs = ads.Analyser.explicitPartialVariantSpecs(dImplSampleSpecs) # Just the same, but less generic.

# c. Add sample order columns (usefull for reports, as pre-analyses are run parallely !).
dfExplSampleSpecs.reset_index(drop=False, inplace=True)
dfExplSampleSpecs.rename(columns=dict(index=sampleNumCol), inplace=True)

# d. Add sample abreviation column (mainly for analysis traces)
dfExplSampleSpecs[sampleAbbrevCol] = dfExplSampleSpecs.apply(sampleAbbreviation, axis='columns')

dfExplSampleSpecs

In [ ]:
computed = False

In [ ]:
workDir = 'tmp/mcds-anlr'

## 4A. Or : Really run analyses

### a. MCDSPreAnalyser object

In [ ]:
anlysr = ads.MCDSPreAnalyser(dfObsIndiv, dfTransects=dfTransects, dSurveyArea=dSurveyArea, abbrevCol=sampleAbbrevCol,
                             resultsHeadCols=dict(before=[sampleNumCol], sample=sampleSelCols,
                                                  after=[sampleAbbrevCol]),
                             transectPlaceCols=transectPlaceCols, passIdCol=passIdCol, effortCol=effortCol,
                             sampleDecCols=sampleDecCols,
                             distanceUnit='Meter', areaUnit='Hectare',
                             surveyType='Point', distanceType='Radial', clustering=False,
                             workDir=workDir)

### b. Run pre-analyses

In [ ]:
# Model fall-down strategy
dModelStrategy = [dict(keyFn=kf, adjSr='COSINE', estCrit='AIC', cvInt=95) \
                  for kf in['HNORMAL', 'HAZARD', 'UNIFORM', 'NEXPON']]

In [ ]:
%%time

# Analyses : min=1.5s, max=3.1s elapsed for 12 samples on a Lenovo P52 (6-core i7-8850H with PCI-e SSD)
results = anlysr.run(dfExplSampleSpecs, dModelStrategy, threads=6)

computed = True

In [ ]:
anlysr.shutdown()

In [ ]:
results.dfTransData('fr')

In [ ]:
results.dfData

### c. Save results for later reload or examination

In [ ]:
results.toExcel(pl.Path(workDir) / 'valtests-mcds-preanlyser-results.xlsx')

In [ ]:
#results.toExcel(pl.Path(workDir) / 'valtests-mcds-preanlyser-results-fr.xlsx', lang='fr')

## 4B. Or : Load analyses from a previous run

(already run and saved above)

In [ ]:
if not computed:
    
    # An analyser object knowns how to build an empty results object ...
    anlysr = ads.MCDSPreAnalyser(dfObsIndiv, dfTransects=dfTransects, dSurveyArea=dSurveyArea,
                                 abbrevCol=sampleAbbrevCol,
                                 resultsHeadCols=dict(before=[sampleNumCol], sample=sampleSelCols,
                                                      after=[sampleAbbrevCol]),
                                 transectPlaceCols=transectPlaceCols, passIdCol=passIdCol, effortCol=effortCol,
                                 sampleDecCols=sampleDecCols,
                                 distanceUnit='Meter', areaUnit='Hectare',
                                 surveyType='Point', distanceType='Radial', clustering=False,
                                 workDir=workDir)
    
    results = anlysr.setupResults()
    
    anlysr.shutdown()

    # Load resultas from file
    resFileName = pl.Path(workDir) / 'valtests-mcds-anlyser-results.xlsx'
    print('Loading results from {} ...'.format(resFileName))

    results.fromExcel(resFileName)
    
else:
    
    print('Just computed, not reloading ...')
    
print('... {} analyses to compare'.format(len(results)))

## 5. Compare results to reference

(reference generated with same kind of "long" code like in III above, but on another data set)

In [ ]:
# Load reference
# 1. Clone results _without_ data.
rsRef = results.copy(withData=False)

# 2. Load it with reference data
rsRef.fromOpenDoc('refout/ACDC2019-Naturalist-ExtraitPreResultats.ods')

rsRef.dfData

In [ ]:
# Compare (ignore sample and analysis indexes, no use here).
indexCols = [col for col in results.miCustomCols.to_list() if '(sample)' in col[0]] \
            + [('parameters', 'estimator key function', 'Value'),
               ('parameters', 'estimator adjustment series', 'Value')]
subsetCols = [col for col in results.dfData.columns.to_list() \
              if col not in (indexCols + [col for col in results.miCustomCols.to_list() if '(sample)' not in col[0]]
                             + [('parameters', 'estimator selection criterion', 'Value'),
                                ('parameters', 'CV interval', 'Value'),
                                ('run output', 'run time', 'Value'),
                                ('run output', 'run folder', 'Value'),
                                ('detection probability', 'key function type', 'Value'),
                                ('detection probability', 'adjustment series type', 'Value')])]

dfDiff = rsRef.compare(results, indexCols=indexCols, subsetCols=subsetCols, dropCloser=13, dropNans=True)

assert dfDiff.empty, 'Oh oh ... some differences !'

print('Yessssss !')

In [ ]:
# To be perfectly honest ... may be some 10**-15 glitches (due to worksheet I/O ?)
rsRef.compare(results, indexCols=indexCols, subsetCols=subsetCols, dropCloser=16, dropNans=True)

# IX. Truncation optimisation (short code + fast run)

Thanks to MCDSZeroOrderTruncationOptimiser class.

In [ ]:
transectPlaceCols = ['Point']
passIdCol = 'Passage'
effortCol = 'Effort'

sampleDistCol = 'Distance'
sampleDecCols = [effortCol, sampleDistCol]

sampleNumCol = 'NumEchant'
sampleSelCols = ['Espèce', passIdCol, 'Adulte', 'Durée']

sampleAbbrevCol = 'AbrevEchant'

dSurveyArea = dict(Zone='ACDC', Surface='2400')

## 1. Individuals data set

In [ ]:
dfObsIndiv = ads.DataSet('refin/ACDC2019-Naturalist-ExtraitObsIndiv.ods', sheet='DonnéesIndiv').dfData
dfObsIndiv

In [ ]:
{ col: dfObsIndiv[col].unique() for col in ['Observateur', 'Point', 'Passage', 'Adulte', 'Durée', 'Espèce'] }

## 2. Actual transects

(can't deduce them from data, some points are missing because of data selection)

In [ ]:
dfTransects = ads.DataSet('refin/ACDC2019-Naturalist-ExtraitObsIndiv.ods', sheet='Inventaires').dfData
dfTransects

## 3. Samples and analyses to optimise

In [ ]:
optIndCol = 'IndOptim'
optAbbrevCol = 'AbrevOptim'

In [ ]:
def optimAbbrev(sAnlys):
    
    # Sample abbreviation
    spcAbbrev = ''.join(word[:4].title() for word in sAnlys['Espèce'].split(' ')[:2])
    sampAbbrev = [str(x) for x in [spcAbbrev, sAnlys.Passage.replace('+', ''),
                                   sAnlys.Adulte.replace('+', ''), sAnlys['Durée']]]

    # Model + Parameters abbreviation
    modParAbbrev = [sAnlys['FonctionClé'][:3].lower(), sAnlys['SérieAjust'][:3].lower()]
    
    return '-'.join(sampAbbrev + modParAbbrev)

In [ ]:
dfOptimExplSpecs = ads.Analyser.explicitVariantSpecs('refin/ACDC2019-Naturalist-ExtraitSpecsAnalyses.xlsx', 
                                                     ignore=['Params3_expl'])

# No use of these cols, as we'll compute them !
dfOptimExplSpecs = dfOptimExplSpecs.drop(columns=['TrGche', 'TrDrte', 'NbTrchMod']).drop_duplicates() \
                    .reset_index(drop=True).reset_index()

dfOptimExplSpecs.rename(columns=dict(index=optIndCol), inplace=True)
dfOptimExplSpecs[optAbbrevCol] = dfOptimExplSpecs.apply(optimAbbrev, axis='columns')
dfOptimExplSpecs

## 4. Add optimisation spec columns

In [ ]:
dfMoreOptimCols = pd.DataFrame([dict(CritChx='AIC', IntervConf=95,
                                     TroncGche='auto', TroncDrte='auto', MethOutliers='tucquant(2.5)',
                                     NbTrModel='mult(2/3, 3/2)', NbTrDiscr=None,
                                     ExprOpt='max(chi2)', MoteurOpt='zoopt(160)')]*len(dfOptimExplSpecs))
#dfMoreOptimCols

In [ ]:
dfOptimExplSpecs = pd.concat([dfOptimExplSpecs, dfMoreOptimCols], axis='columns')
dfOptimExplSpecs

In [ ]:
computed = False

## 5A. Or : Really run optimisations

### a. MCDS Zeroth Order Truncation Optimiser object

In [ ]:
dfObsIndiv.head()

In [ ]:
# Colonnes de dfOptimExplSpecs donnant les paramètres d'analyse / optimisation
optimParamsSpecsCols  = ['FonctionClé', 'SérieAjust', 'CritChx', 'IntervConf',
                         'TroncGche', 'TroncDrte', 'MethOutliers', 'NbTrModel', 'NbTrDiscr',
                         'ExprOpt', 'MoteurOpt']

In [ ]:
zoptr = ads.MCDSZerothOrderTruncationOptimiser \
                (dfObsIndiv, effortConstVal=1, dSurveyArea=dSurveyArea, 
                 transectPlaceCols=transectPlaceCols, passIdCol=passIdCol, effortCol=effortCol,
                 sampleDecCols=sampleDecCols, sampleDistanceCol=sampleDistCol,
                 distanceUnit='Meter', areaUnit='Hectare',
                 surveyType='Point', distanceType='Radial', clustering=False,
                 resultsHeadCols=dict(before=[optIndCol], sample=sampleSelCols, after=optimParamsSpecsCols),
                 abbrevCol=optAbbrevCol, workDir='c:/tmp', logData=False,                 
                 defCoreMaxIters=120)

### b. Run analyses

In [ ]:
assert zoptr.checkUserSpecs(dfOptimExplSpecs, optimParamsSpecsCols)

In [ ]:
%%time

# Analyses
results = zoptr.run(dfOptimExplSpecs, optimParamsSpecsCols, threads=6)

computed = True

12 analyses specs (6 parallel threads)

CritChx='AIC', IntervConf=95, TroncGche='auto', TroncDrte='auto', MethOutliers='tucquant(2.5)',
NbTrModel='mult(2/3, 3/2)', NbTrDiscr=None, ExprOpt='max(chi2)'

MoteurOpt='zoopt(160)'

2020-06-29 21:18:04,727 Wall time: 4min 31s

MoteurOpt='zoopt(250, tv=0.6)'

2020-06-28 19:23:38,868 Wall time: 9min 19s

In [ ]:
zoptr.shutdown()

In [ ]:
results.dfTransData('fr')

### c. Save results for later reload or examination

In [ ]:
results.toExcel('tmp/valtests-mcds-optimiser-results.xlsx')

In [ ]:
#results.toExcel(pl.Path(workDir) / 'valtests-mcds-anlyser-results-fr.xlsx', lang='fr')

In [ ]:
?????????????????????????????????????????????? ... to be continued ...

## 4B. Or : Load analyses from a previous run

(already run and saved above)

In [ ]:
if not computed:
    
    # An analyser object knowns how to build an empty results object ...
    anlysr = ads.MCDSAnalyser(dfObsIndiv, dfTransects=dfTransects, dSurveyArea=dSurveyArea, abbrevCol=anlysAbbrevCol,
                              resultsHeadCols=dict(before=[varIndCol, sampleNumCol], sample=sampleSelCols,
                                                   after=[anlysAbbrevCol]),
                              transectPlaceCols=transectPlaceCols, passIdCol=passIdCol, effortCol=effortCol,
                              sampleDecCols=sampleDecCols,
                              distanceUnit='Meter', areaUnit='Hectare',
                              surveyType='Point', distanceType='Radial', clustering=False,
                              workDir=workDir)
    
    results = anlysr.setupResults()
    
    anlysr.shutdown()
    
    # Load results from file.
    resFileName = 'c:/tmp/valtests-mcds-anlyser-results.xlsx'
    print('Loading results from {} ...'.format(resFileName))

    results.fromExcel(resFileName)
    
else:
    
    print('Just computed, not reloading ...')
    
print('... {} analyses to compare'.format(len(results)))

## 5. Compare results to reference

(reference generated with same kind of "long" code like in III above, but on another data set)

In [ ]:
# Load reference
# 1. Clone results _without_ data.
rsRef = results.copy(withData=False)

# 2. Load it with reference data
rsRef.fromOpenDoc('refout/ACDC2019-Naturalist-ExtraitResultats.ods')

rsRef.dfData

In [ ]:
# Compare.
indexCols = results.miCustomCols.to_list() + [('parameters', 'estimator key function', 'Value'),
                                              ('parameters', 'estimator adjustment series', 'Value'),
                                              ('parameters', 'left truncation distance', 'Value'),
                                              ('parameters', 'right truncation distance', 'Value'),
                                              ('parameters', 'model fitting distance cut points', 'Value')]
subsetCols = [col for col in results.dfData.columns.to_list() \
              if col not in (indexCols + [('parameters', 'estimator selection criterion', 'Value'),
                                          ('parameters', 'CV interval', 'Value'),
                                          ('run output', 'run time', 'Value'),
                                          ('run output', 'run folder', 'Value'),
                                          ('detection probability', 'key function type', 'Value'),
                                          ('detection probability', 'adjustment series type', 'Value')])]

dfDiff = rsRef.compare(results, indexCols=indexCols, subsetCols=subsetCols, dropCloser=13, dropNans=True)

assert dfDiff.empty

print('Yessssss !')

In [ ]:
# To be perfectly honnest ... some 10**-15 glitches (due to worksheet I/O ?)
rsRef.compare(results, indexCols=indexCols, subsetCols=subsetCols, dropCloser=14, dropNans=True)

# X. Truncation optimisation : Statistics on parameter variants

Objective: How to choose key parameters ?
* how many outliers ?
* how many max iters ?
* correlation with number of sightings ?

## 1. Data set, samples, transects, analyses

In [ ]:
# First, run IX. above up to 3.

## 2. Parameter variants plan

In [ ]:
nTimes = 2 #20

In [ ]:
expr2MaxPlan = ['chi2'] #, 'ks', 'chi2*ks']

In [ ]:
outliersPctPlan = [2.5, 5.0]

In [ ]:
maxItersPlan = [10] # [50, 100, 150, 200, 250, 300]

In [ ]:
computed = False

## 3A. Or: Run optimisations according to the plan

In [ ]:
# Colonnes de dfOptimExplSpecs donnant les paramètres d'analyse / optimisation
optimParamsSpecsCols  = ['FonctionClé', 'SérieAjust', 'CritChx', 'IntervConf',
                         'TroncGche', 'TroncDrte', 'MethOutliers', 'NbTrModel', 'NbTrDiscr',
                         'ExprOpt', 'ParExec', 'MoteurOpt']

In [ ]:
%%time

ldfResults = list()

for expr2Max in expr2MaxPlan:
    
    for olrsPct in outliersPctPlan:

        for maxIters in maxItersPlan:

            logger.info(f'Params set: {expr2Max=}, {nTimes=}, {maxIters=}, {olrsPct=:.1f}')
            
            # Prepare optim. params.
            dfMoreOptimCols = \
                pd.DataFrame([dict(CritChx='AIC', IntervConf=95,
                                   TroncGche='auto', TroncDrte='auto',
                                   MethOutliers=f'tucquant({olrsPct:.1f})',
                                   NbTrModel='mult(2/3, 3/2)', NbTrDiscr=None,
                                   ExprOpt=f'max({expr2Max})', ParExec=f'times({nTimes})',
                                   MoteurOpt=f'zoopt({maxIters})')]*len(dfOptimExplSpecs))

            dfOptVarExplSpecs = pd.concat([dfOptimExplSpecs, dfMoreOptimCols], axis='columns')

            # Run optimisation.
            zoptr = ads.MCDSZerothOrderTruncationOptimiser \
                            (dfObsIndiv, effortConstVal=1, dSurveyArea=dSurveyArea, 
                             transectPlaceCols=transectPlaceCols, passIdCol=passIdCol, effortCol=effortCol,
                             sampleDecCols=sampleDecCols, sampleDistanceCol=sampleDistCol,
                             distanceUnit='Meter', areaUnit='Hectare',
                             surveyType='Point', distanceType='Radial', clustering=False,
                             resultsHeadCols=dict(before=[optIndCol], sample=sampleSelCols, after=optimParamsSpecsCols),
                             abbrevCol=optAbbrevCol, workDir='tmp/mcds-optstats', logData=False,                 
                             defCoreMaxIters=120)

            assert zoptr.checkUserSpecs(dfOptVarExplSpecs, optimParamsSpecsCols)

            results = zoptr.run(dfOptVarExplSpecs, optimParamsSpecsCols, threads=6)

            zoptr.shutdown()

            # Save results for this run
            ldfResults.append(results.dfData)
        
# Done : concat and save results.
dfResults = pd.concat(ldfResults, ignore_index=True)

resFileName = 'tmp/valtests-mcds-opter-res4stats.xlsx'
dfResults.to_excel(resFileName, index=False)
logger.info(f'Results saved to {resFileName}')

computed = True

## 3B. Or : Load results from a previous run

(already run and saved above)

In [ ]:
if not computed:
    
    # Load results from file.
    resFileName = 'tmp/valtests-mcds-opter-res4stats.xlsx'
    print('Loading results from {} ...'.format(resFileName))

    dfResults = pd.read_excel(resFileName)
    
else:
    
    print('Just computed, not reloading ...')
    
print('... {} results to process'.format(len(dfResults)))

## 4. Compute stats on results

In [ ]:
optResCols = ['minDist', 'maxDist', 'fitDistCuts', 'chi2', 'ks', 'chi2*ks']
groupCols = [col for col in dfResults.columns if col not in optResCols]

In [ ]:
dfResults.groupby(groupCols).agg(['mean', 'std'])

In [ ]:
dfResults.columns

In [ ]:
?????????????????? ... to be continued ... data ready : valtests-mcds-opter-res4stats.xlsx

# Development

# Sandbox